<a href="https://colab.research.google.com/github/Kjgwo/BigData/blob/main/BigData5%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BeautifulSoup

In [ ]:
import requests
crawling_url = 'http://www.hanbit.co.kr/store/books/full_book_list.html'
response = requests.get(crawling_url)
text = response.text

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(text, 'html.parser')

In [ ]:
result = soup.find_all('tr')

In [ ]:
urls = []
titles = []
authors = []
dates = []
brands = []
prices = []

for item in result:
    it1 = item.find_all('a')
    it2 = item.find_all('td')
    for it in it1:
        url = it.get('href')
        if(url == None):
            continue
        url = "https://www.hanbit.co.kr" + url
        urls.append(url)

        brands.append(it2[0].string)
        titles.append(it2[1].string)
        authors.append(it2[2].string)
        dates.append(it2[3].string)
        prices.append(it2[4].string)

In [ ]:
import pandas as pd
df = pd.DataFrame(
    {
        "BRAND": brands,
        "TITLE": titles,
        "AUTHOR": authors,
        "DATE": dates,
        "PRICE": prices
    }
)
df

In [ ]:
import requests
crawling_url = 'http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?orderClick=d79'
response = requests.get(crawling_url)
text = response.text

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(text, 'html.parser')

In [ ]:
date = soup.find('h4', {'class': 'title_best_basic'}).find('small').text
print(date)